In [6]:
!pip install vllm pandas joblib

In [1]:
from vllm import LLM, SamplingParams
import textwrap
import os
import json
import time
import requests
import pandas as pd
from itertools import combinations, product
from tqdm.auto import tqdm
from joblib import Parallel, delayed

INFO 08-12 11:54:16 [__init__.py:235] Automatically detected platform cuda.


In [2]:
BLOOM_LEVELS = ["จำ", "เข้าใจ", "นำไปใช้", "วิเคราะห์", "ประเมิน", "สร้างสรรค์"]
LEVELS = ["ง่าย", "ปานกลาง", "ยาก"]
GRADE_LEVELS = ["ม.4", "ม.5", "ม.6"]
TOPICS = ["พีชคณิต"]
QUESTION_TYPES = ["multiple_choice"]

N_JOBS = 8

In [3]:
bloom_combinations = [[b] for b in BLOOM_LEVELS] + [list(pair) for pair in combinations(BLOOM_LEVELS, 2)]
all_combinations = list(product(TOPICS, GRADE_LEVELS, QUESTION_TYPES, LEVELS, bloom_combinations))
print(f"Total combinations: {len(all_combinations)}")

Total combinations: 189


In [4]:
def create_user_prompt(topic, grade_level, question_type, difficulty, bloom_levels, num_problems=1):
    bloom_str = ", ".join(bloom_levels)
    prompt = f"""จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้
1. หัวข้อ: {topic}
2. สำหรับนักเรียน: {grade_level}
3. รูปแบบ: {question_type}
4. ความยาก: {difficulty}
5. bloom level: {bloom_str}
6. จำนวน: 1 ข้อ
7. เพิ่มเติม: โจทย์จำเป็นต้องมีคำตอบ และถ้าโจทย์เป็นแบบ multiple choice (ปรนัย) ต้องมีคำตอบหลอกจำนวน 3 ข้อ (ทั้งหมด หลอก + จริง มี 4 ข้อ) โดยมาจากการคำนวนที่ผิดพลาด"""
    return prompt

In [5]:
llm = LLM(model="Qwen/Qwen3-4B-Thinking-2507", gpu_memory_utilization=0.95, max_model_len=32768)

INFO 08-12 11:54:26 [config.py:1604] Using max model len 32768
INFO 08-12 11:54:27 [config.py:2434] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 08-12 11:54:28 [core.py:572] Waiting for init message from front-end.
INFO 08-12 11:54:28 [core.py:71] Initializing a V1 LLM engine (v0.10.0) with config: model='Qwen/Qwen3-4B-Thinking-2507', speculative_config=None, tokenizer='Qwen/Qwen3-4B-Thinking-2507', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observability_config=Observabil

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 08-12 11:54:32 [default_loader.py:262] Loading weights took 1.23 seconds
INFO 08-12 11:54:33 [gpu_model_runner.py:1892] Model loading took 7.6065 GiB and 1.647040 seconds
INFO 08-12 11:54:40 [backends.py:530] Using cache directory: /root/.cache/vllm/torch_compile_cache/e2dfbc83f1/rank_0_0/backbone for vLLM's torch.compile
INFO 08-12 11:54:40 [backends.py:541] Dynamo bytecode transform time: 6.47 s
INFO 08-12 11:54:44 [backends.py:194] Cache the graph for dynamic shape for later use
INFO 08-12 11:55:09 [backends.py:215] Compiling a graph for dynamic shape takes 28.31 s
INFO 08-12 11:55:22 [monitor.py:34] torch.compile takes 34.78 s in total
INFO 08-12 11:55:24 [gpu_worker.py:255] Available KV cache memory: 61.85 GiB
INFO 08-12 11:55:24 [kv_cache_utils.py:833] GPU KV cache size: 450,368 tokens
INFO 08-12 11:55:24 [kv_cache_utils.py:837] Maximum concurrency for 32,768 tokens per request: 13.74x


Capturing CUDA graph shapes: 100%|██████████| 67/67 [00:02<00:00, 26.95it/s]


INFO 08-12 11:55:27 [gpu_model_runner.py:2485] Graph capturing finished in 3 secs, took 0.76 GiB
INFO 08-12 11:55:27 [core.py:193] init engine (profile, create kv cache, warmup model) took 54.05 seconds


In [8]:
def call_llm(user_prompt, system_prompt=SYSTEM_PROMPT):
    data = [
      {"role": "system", "content": SYSTEM_PROMPT},
      {"role": "user", "content": user_prompt}
    ]

    full_prompt = f"{system_prompt}\n\nUser: {user_prompt}\nAssistant:"

    sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=8192)
    output = llm.generate(full_prompt, sampling_params)

    for output_item in output:
        prompt = output_item.prompt
        generated_text = output_item.outputs[0].text
        # wrapped_text = textwrap.fill(generated_text, width=200)
        # print(f"{wrapped_text}")
        return generated_text

In [9]:
def extract_think_and_content(output_str):
    think, content = "", ""
    if "<think>" in output_str and "</think>" in output_str:
        think = output_str.split("<think>")[1].split("</think>")[0].replace("\n", " ").strip()
    if "<questions>" in output_str:
        content = output_str.split("<questions>", 1)[1]
        content = "<questions>" + content
        content = content.replace("\n", " ").strip()
    return think, content


def save_result(think, content, params, result_meta=None, outdir="results"):
    os.makedirs(outdir, exist_ok=True)
    run_id = int(time.time())
    record = {
        "topic": params[0],
        "grade": params[1],
        "qtype": params[2],
        "level": params[3],
        "bloom": params[4],
        "think": think,
        "content": content,
    }
    with open(f"{outdir}/result_{run_id}.json", "w", encoding="utf-8") as f:
        json.dump(record, f, ensure_ascii=False)

In [10]:
# full_prompt = f"{SYSTEM_PROMPT}\n\nUser: {USER_PROMPT}\nAssistant:"

# sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=8192)
# output = llm.generate(full_prompt, sampling_params)

# for output_item in output:
#     prompt = output_item.prompt
#     generated_text = output_item.outputs[0].text
#     wrapped_text = textwrap.fill(generated_text, width=80)
#     print(f"{wrapped_text}")

# result = call_llm(create_user_prompt(*all_combinations[0]))

def _first_text_from_result(result: dict) -> str:
    try:
        out = result.get("output", [])
        if not out:
            return ""
        choices = out[0].get("choices", [])
        if choices:
            msg = choices[0]
            tokens = msg.get("tokens")
            if isinstance(tokens, list):
                return "".join(tokens)
            content = msg.get("message", {}).get("content")
            if isinstance(content, str):
                return content
        return json.dumps(out[0], ensure_ascii=False)
    except Exception:
        return ""


def process_one_combination(params, outdir="results"):
    topic, grade, qtype, level, bloom = params
    user_prompt = create_user_prompt(topic, grade, qtype, level, bloom)
    result = call_llm(user_prompt)
    if not result:
        return None

    think, content = extract_think_and_content(result)

    # Save as JSON per sample
    meta = {
        k: result.get(k) for k in ("delayTime", "executionTime", "status", "workerId") if k in result
    }
    save_result(think, content, params=params, result_meta=meta, outdir=outdir)
    return {
        "topic": topic,
        "grade": grade,
        "qtype": qtype,
        "level": level,
        "bloom": bloom,
        "think": think,
        "content": content
    }

In [11]:
# ทดสอบระบบ (full system test 1 รอบ)
test_result = process_one_combination(all_combinations[0], outdir="test_results")
print("Test result:", test_result)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Test result: {'topic': 'พีชคณิต', 'grade': 'ม.4', 'qtype': 'multiple_choice', 'level': 'ง่าย', 'bloom': ['จำ'], 'think': 'Okay, I need to create a high-quality math problem for 4th grade high school students (m.4) in algebra, multiple choice format, easy difficulty, Bloom\'s level "จำ" (Remember). Let me think step by step.  First, Bloom\'s level "จำ" means the question is about recalling facts, formulas, or basic steps. So the question should test the student\'s ability to remember a formula or a basic concept without needing to apply it.  The user specified multiple choice with 4 options, 1 correct and 3 distractors (wrong answers) that come from common calculation mistakes.  For algebra in m.4, common topics might include linear equations, basic algebraic expressions, solving for variables, maybe the quadratic formula? Wait, but m.4 is typically around 10th grade (Grade 10) in some systems. Wait, in Thailand, ม.4 is equivalent to Grade 10. So they\'ve probably covered linear equatio

In [12]:
# Full system run (parallel) with tqdm progress
N_JOBS = 9
results = []
for res in tqdm(
    Parallel(n_jobs=N_JOBS, prefer="threads", return_as="generator")(
        delayed(process_one_combination)(params) for params in all_combinations
    ),
    total=len(all_combinations),
    desc="Generating"
):
    results.append(res)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Generating:   0%|          | 0/189 [00:00<?, ?it/s]

...

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

KeyboardInterrupt: 

In [15]:
# Save
results = [r for r in results if r]
df = pd.DataFrame(results)
df.to_csv("all_results.csv", index=False, encoding="utf-8-sig")
# df.to_parquet("all_results.parquet", index=False)

In [16]:
import zipfile
    
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), 
                       os.path.relpath(os.path.join(root, file), 
                                       os.path.join(path, '..')))

with zipfile.ZipFile('qwen3-4b-thinking-2507.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipdir('export/', zipf)

In [7]:
SYSTEM_PROMPT = """
คุณคือ ดร.อเล็กซานเดอร์ โอเลอร์ นักคณิตศาสตร์อันดับหนึ่งของโลกที่มีชื่อเสียงระดับโลกในด้านการวิจัยคณิตศาสตร์ขั้นสูง คุณได้รับรางวัลฟิลด์สเมดัลและมีผลงานตีพิมพ์ในวารสารคณิตศาสตร์ชั้นนำกว่า 200 บทความ คุณเชี่ยวชาญในทุกสาขาของคณิตศาสตร์ตั้งแต่พีชคณิต เรขาคณิต แคลคูลัส ไปจนถึงทฤษฎีจำนวนและคณิตศาสตร์ประยุกต์

ปัจจุบัน ดร.อเล็กซานเดอร์ กำลังทำหน้าที่เป็นครูสอนวิชาคณิตศาสตร์ที่โรงเรียนมัธยมปลายชื่อดังแห่งหนึ่ง โดยรับผิดชอบการสอนนักเรียนระดับชั้นมัธยมศึกษาปีที่ 4, 5 และ 6 (เทียบเท่า Grade 10, 11, 12) คุณมีปรัชญาในการสอนที่เชื่อว่าการเรียนรู้คณิตศาสตร์ต้องเป็นไปตามลำดับขั้นของการคิดที่เรียกว่า Bloom's Taxonomy เพื่อให้นักเรียนพัฒนาทักษะการคิดอย่างเป็นระบบ

หน้าที่หลักของคุณคือการสร้างโจทย์คณิตศาสตร์ที่มีคุณภาพสูง เหมาะสมกับระดับความสามารถของนักเรียน และสอดคล้องกับหลักการของ Bloom's Taxonomy อย่างเข้มงวด

## Bloom's Taxonomy ในการสอนคณิตศาสตร์

### 1. จำ (Remember)
**ความหมาย**: การจำและการเรียกคืนข้อมูล สูตร หรือขั้นตอนพื้นฐานที่ได้เรียนมาแล้ว
**คำสำคัญ**: จำได้, ระบุ, รายการ, ตั้งชื่อ, เลือก

### 2. เข้าใจ (Understand)
**ความหมาย**: การเข้าใจความหมาย แนวคิด และสามารถอธิบายด้วยคำพูดของตนเองได้
**คำสำคัญ**: อธิบาย, แปล, สรุป, เปรียบเทียบ, แสดงให้เห็น

### 3. นำไปใช้ (Apply)
**ความหมาย**: การนำความรู้ สูตร หรือขั้นตอนที่เรียนมาไปใช้แก้ปัญหาในสถานการณ์ใหม่
**คำสำคัญ**: คำนวณ, แก้, ใช้, แสดง, ดำเนินการ

### 4. วิเคราะห์ (Analyze)
**ความหมาย**: การแยกแยะองค์ประกอบ วิเคราะห์ความสัมพันธ์ และเข้าใจโครงสร้างของปัญหา
**คำสำคัญ**: แยกแยะ, เปรียบเทียบ, ตรวจสอบ, ทดสอบ, วิเคราะห์

### 5. ประเมิน (Evaluate)
**ความหมาย**: การตัดสิน ประเมินค่า ให้เหตุผล และแสดงความคิดเห็นโดยใช้เกณฑ์ที่กำหนด
**คำสำคัญ**: ตัดสิน, ประเมิน, วิจารณ์, แสดงความคิดเห็น, ให้เหตุผล

### 6. สร้างสรรค์ (Create)
**ความหมาย**: การสร้างสรรค์สิ่งใหม่ ออกแบบ วางแผน หรือสร้างโจทย์ปัญหาขึ้นมาเอง
**คำสำคัญ**: สร้าง, ออกแบบ, วางแผน, เสนอ, พัฒนา

## ตัวอย่างโจทย์ตาม Bloom's Taxonomy

### จำ (Remember)
โจทย์: สูตรหาพื้นที่วงกลมคือข้อใด
A) $ A = \pi r^2 $  B) $ A = 2\pi r $  C) $ A = \pi d $  D) $ A = \frac{1}{2}\pi r^2 $

### เข้าใจ (Understand)
โจทย์: อธิบายความหมายของ $ \frac{dy}{dx} = 3x^2 $ ในทางเรขาคณิต

### นำไปใช้ (Apply)
โจทย์: จงหาค่า $ x $ จากสมการ $ 2x + 5 = 13 $

### วิเคราะห์ (Analyze)
โจทย์: เปรียบเทียบพฤติกรรมของฟังก์ชัน $ f(x) = x^2 $ และ $ g(x) = x^3 $ เมื่อ $ x > 1 $

### ประเมิน (Evaluate)
โจทย์: นักเรียนคนหนึ่งอ้างว่า $ \sqrt{a + b} = \sqrt{a} + \sqrt{b} $ เสมอ จงประเมินว่าข้อความนี้ถูกหรือผิด พร้อมให้เหตุผล

### สร้างสรรค์ (Create)
โจทย์: ออกแบบฟังก์ชันกำลังสองที่มีจุดยอดอยู่ที่ $ (2, -3) $ และผ่านจุด $ (0, 1) $

## ตัวอย่างโจทย์แบบผสม 5 โจทย์

### โจทย์ที่ 1 (เข้าใจ + นำไปใช้)
หากฟังก์ชัน $ f(x) = 2x - 3 $ จงหาค่า $ f(5) $ และอธิบายความหมายของผลลัพธ์

### โจทย์ที่ 2 (วิเคราะห์ + ประเมิน)
เปรียบเทียบวิธีแก้สมการ $ x^2 - 5x + 6 = 0 $ ด้วยการแยกตัวประกอบและสูตรกำลังสอง แล้วประเมินว่าวิธีใดมีประสิทธิภาพมากกว่า

### โจทย์ที่ 3 (นำไปใช้ + วิเคราะห์)
ร้านค้าแห่งหนึ่งขายสินค้าในราคา $ 100x - x^2 $ บาท เมื่อขาย $ x $ ชิ้น จงหาจำนวนสินค้าที่ขายได้เงินสูงสุด

### โจทย์ที่ 4 (เข้าใจ + สร้างสรรค์)
จากกราฟ $ y = \sin x $ จงสร้างฟังก์ชันใหม่ที่มีแอมพลิจูดเป็น 3 เท่า และอธิบายการเปลี่ยนแปลง

### โจทย์ที่ 5 (วิเคราะห์ + ประเมิน + สร้างสรรค์)
นักเรียนสองคนแก้โจทย์หาค่าสูงสุดของ $ f(x) = -x^2 + 4x + 1 $ ได้คำตอบต่างกัน คนหนึ่งได้ $ x = 2 $ อีกคนได้ $ x = -2 $ จงวิเคราะห์ว่าใครถูก ประเมินข้อผิดพลาด และสร้างวิธีตรวจสอบคำตอบ

---

## คำสั่งการทำงาน

เมื่อได้รับโจทย์ให้สร้างข้อสอบคณิตศาสตร์ ให้ทำตามขั้นตอนดังนี้:

### ขั้นตอนที่ 1: การคิดและวางแผน
ในแท็ก `<think></think>` ให้ทำการ:
1. **คิดโจทย์เป็นภาษาอังกฤษก่อน** - เพื่อให้ได้แนวคิดที่ชัดเจน
2. **ทดลองหาคำตอบ** - ตรวจสอบว่าโจทย์สามารถแก้ได้จริง
3. **หากแก้ไม่ได้ ให้เปลี่ยนโจทย์** - แล้วทดลองใหม่จนได้โจทย์ที่ดี
4. **เมื่อได้โจทย์ที่สมบูรณ์แล้ว** - สร้างตัวเลือกคำตอบสำหรับ multiple choice (หากต้องการ)
5. **ตรวจสอบความสอดคล้องกับ Bloom's taxonomy** ที่กำหนด

### ขั้นตอนที่ 2: การเขียนผลลัพธ์
หลังจาก `<think></think>` แล้ว ให้เขียนโจทย์ในรูปแบบ XML ตามตัวอย่างนี้:

```xml
<questions>
  <question>
    <text>โจทย์คณิตศาสตร์ที่มี KaTeX formatting เช่น $ 4x + 3 = 2x + 9 $</text>
    <type>multiple_choice</type>
    <options>
      <option>$ ตัวเลือก1 $</option>
      <option>$ ตัวเลือก2 $</option>
      <option>$ ตัวเลือก3 $</option>
      <option>$ ตัวเลือก4 $</option>
    </options>
    <correct_answer>$ คำตอบที่ถูก $</correct_answer>
    <explanation>
      ขั้นตอนที่ 1: อธิบายด้วย KaTeX เช่น $ 4x - 2x + 3 = 9 $
      ขั้นตอนที่ 2: $ 2x + 3 = 9 $
      ขั้นตอนที่ 3: $ 2x = 6 $
      ขั้นตอนที่ 4: $ x = 3 $
    </explanation>
    <score>2</score>
    <difficulty>ง่าย</difficulty>
    <bloom_levels>
      <level>เข้าใจ</level>
      <level>วิเคราะห์</level>
    </bloom_levels>
  </question>

  <question>
    <text>จงหาค่า $ x $ ที่ทำให้ $ 3x - 7 = 2x + 5 $</text>
    <type>short_answer</type>
    <correct_answer>$ x = 12 $</correct_answer>
    <explanation>
      ขั้นตอนที่ 1: นำ $ 2x $ ไปยังข้างซ้าย และ $ 7 $ ไปยังข้างขวา
      $ 3x - 2x = 5 + 7 $
      ขั้นตอนที่ 2: คำนวณ
      $ x = 12 $
    </explanation>
    <score>2</score>
    <difficulty>ปานกลาง</difficulty>
    <bloom_levels>
      <level>เข้าใจ</level>
    </bloom_levels>
  </question>
</questions>
```

## หมายเหตุสำคัญ

### รูปแบบการเขียน:
- **ใช้ KaTeX format** `$ ... $` สำหรับตัวเลข สูตร และนิพจน์คณิตศาสตร์ทั้งหมด
- **ใช้คำไทยสำหรับ Bloom's level**: จำ, เข้าใจ, นำไปใช้, วิเคราะห์, ประเมิน, สร้างสรรค์
- **สามารถมีหลายระดับ Bloom's**: `<bloom_levels><level>เข้าใจ</level><level>นำไปใช้</level></bloom_levels>`

### ระดับความยาก:
- **ง่าย**: โจทย์พื้นฐาน 1-2 ขั้นตอน (คะแนน 1-2)
- **ปานกลาง**: โจทย์ที่ต้องใช้ความคิด 3-4 ขั้นตอน (คะแนน 3-4)
- **ยาก**: โจทย์ซับซ้อน ต้องวิเคราะห์เชิงลึก 5+ ขั้นตอน (คะแนน 5+)

### การให้คะแนน:
- Score ควรสะท้อนระดับความยากและเวลาที่ใช้ในการแก้ปัญหา
- โจทย์ที่ต้องใช้ Bloom's level สูงควรได้คะแนนมากกว่า

คุณพร้อมที่จะสร้างโจทย์คณิตศาสตร์คุณภาพสูงตามที่ได้รับมอบหมายแล้วหรือไม่?"""

USER_PROMPT = """
จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้
1. หัวข้อ: พีชคณิต
2. สำหรับนักเรียน: ม.4
3. รูปแบบ: multiple_choice
4. ความยาก: ยาก
5. bloom level: เข้าใจ, นำไปใช้
6. จำนวน: 1 ข้อ
7. เพิ่มเติม: โจทย์จำเป็นต้องมีคำตอบ และถ้าโจทย์เป็นแบบ multiple choice (ปรนัย) ต้องมีคำตอบหลอกจำนวน 3 ข้อ (ทั้งหมด หลอก + จริง มี 4 ข้อ) โดยมาจากการคำนวนที่ผิดพลาด
"""